# IMT 574 Problem Set 5

Loading Libraries

In [1]:
#IPython is what you are using now to run the notebook
import IPython
print( "IPython version:      %6.6s (need at least 1.0)" % IPython.__version__)

# Numpy is a library for working with arrays and matrices
import numpy as np
print( "Numpy version:        %6.6s (need at least 1.7.1)" % np.__version__)

# SciPy implements many different numerical algorithms
import scipy as sp
print( "SciPy version:        %6.6s (need at least 0.12.0)" % sp.__version__)

# Pandas makes working with data tables easier
import pandas as pd
print( "Pandas version:       %6.6s (need at least 0.11.0)" % pd.__version__)

# Module for plotting
import matplotlib.pyplot as plt  
from pylab import *
print( "Matpltolib version:    %6.6s (need at least 1.2.1)" %
       matplotlib.__version__)
%matplotlib inline
# necessary for in-line graphics

# SciKit Learn implements several Machine Learning algorithms
import sklearn
print("Scikit-Learn version: %6.6s (need at least 0.13.1)" %
       sklearn.__version__)
import os
# for certain system-related functions

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score

IPython version:       7.8.0 (need at least 1.0)
Numpy version:        1.16.5 (need at least 1.7.1)
SciPy version:         1.3.1 (need at least 0.12.0)
Pandas version:       0.25.1 (need at least 0.11.0)
Matpltolib version:     3.1.1 (need at least 1.2.1)
Scikit-Learn version: 0.21.3 (need at least 0.13.1)


# 1 Explore and clean the data

1. Take a look at a few lines of data (you may use pd.sample for this).

In [2]:
movie_df= pd.read_csv("rotten-tomatoes.csv.bz2")
print(movie_df.shape)
movie_df.sample(5)

(13442, 9)


,critic,fresh,imdb,link,publication,quote,review_date,rtid,title
11567,Frank S. Nugent,fresh,31762,http://movies.nytimes.com/movie/review?res=9E0...,New York Times,"It's a fairly good melodrama, nothing more.",2003-05-20 00:00:00,19047,Only Angels Have Wings
11591,Variety Staff,fresh,63823,http://www.variety.com/review/VE1117796524.htm...,Variety,Here are all the ingredients of a novel entert...,2007-09-06 00:00:00,9408,Yellow Submarine
10419,Roger Ebert,rotten,141098,http://www.rogerebert.com/reviews/forces-of-na...,Chicago Sun-Times,"A romantic shaggy dog story, a movie that lead...",2000-01-01 00:00:00,11350,Forces of Nature
1449,James Berardinelli,rotten,114594,http://www.reelviews.net/movies/s/swimming.html,ReelViews,There doesn't seem to be enough material here ...,2000-01-01 00:00:00,15161,Swimming with Sharks
10965,Owen Gleiberman,fresh,99052,"http://www.ew.com/ew/article/0,,317842,00.html",Entertainment Weekly,"It gives you the willies in a cheery, presenta...",1990-07-18 00:00:00,11047,Arachnophobia


2. Print out all variable names.

In [3]:
movie_df.columns

Index(['critic', 'fresh', 'imdb', 'link', 'publication', 'quote',
       'review_date', 'rtid', 'title'],
      dtype='object')

3. Create a summary table (maybe more like a bullet list) where you print out the most important summary statistics for the most interesting variables. The most interesting facts you should present should include: a) number of missings for fresh and quote; b) all different values for fresh/rotten evaluations; c) counts or percentages of these values; d) number of zero-length or only whitespace quote-s; e) minimum-maximum-average length of quotes (either in words, or in characters). (Can you do this as an one-liner?); f) how many reviews are in data multiple times. Feel free to add more figures you consider relevant.

In [4]:
null_fresh=movie_df.fresh.isna().sum()
null_quote=movie_df.quote.isna().sum()
diff_val=movie_df.fresh.unique()
per_fresh=round((movie_df.fresh=="fresh").sum()*100/len(movie_df.fresh),2)
per_rot=round((movie_df.fresh=="rotten").sum()*100/len(movie_df.fresh),2)
per_none=round((movie_df.fresh=="none").sum()*100/len(movie_df.fresh),2)
num_zero= movie_df.quote.isnull().sum() + movie_df[movie_df.quote.str.len()==0].shape[0]
num_white_space = round(movie_df.quote==" ").sum() + movie_df[movie_df.quote.str.isspace()].shape[0]
repeat_quotes = movie_df.quote.shape[0] + movie_df.quote.unique().size
max_quote = movie_df.quote.map(lambda x: len(x)).max()
min_quote = movie_df.quote.map(lambda x: len(x)).min()
avg_quote = round(movie_df.quote.map(lambda x: len(x)).mean(),2)

In [5]:
print("Number of missing values for the 'fresh' variable is {:d}".format(null_fresh))
print("Number of missing values for the 'quote' variable is {:d}".format(null_quote))
print("All different values for fresh/rotten evaluations are", diff_val[0]+ ",",diff_val[1], "and",diff_val[2]+".")
print("Percentage of fresh values is",per_fresh,"%")
print("Percentage of rotten values is",per_rot, "%")
print("Percentage of none values is", per_none,"%")
print("Number of zero-length quotes is", num_zero)
print("Number of only whitespace quotes is", num_white_space)
print("Minimum length of quotes (in characters) is", min_quote)
print("Maximum length of quotes (in characters) is", max_quote)
print("Average length of quotes (in characters) is", avg_quote)
print("Number of reviews present multiple times in the data is ",repeat_quotes)

Number of missing values for the 'fresh' variable is 0
Number of missing values for the 'quote' variable is 0
All different values for fresh/rotten evaluations are fresh, rotten and none.
Percentage of fresh values is 62.41 %
Percentage of rotten values is 37.42 %
Percentage of none values is 0.17 %
Number of zero-length quotes is 0
Number of only whitespace quotes is 0.0
Minimum length of quotes (in characters) is 4
Maximum length of quotes (in characters) is 256
Average length of quotes (in characters) is 121.23
Number of reviews present multiple times in the data is  26278


4. Now when you have an overview what you have in data, clean it by removing all the inconsistencies the table reveals. We have to ensure that the central variables, quote and fresh are not missing, quote is not an empty string (or just contain spaces and such), and all rows are unique.

In [6]:
movie_df_clean=movie_df.drop_duplicates() #dropping duplicate data
movie_df_new=movie_df_clean[movie_df_clean.fresh!="none"] #removing reviews which are neither fresh nor rotten

# 2 Naive Bayes

2. Convert your data (quotes) into bag-of-words. Your code should look something like this:

In [7]:
# define vectorizer
vectorizer = CountVectorizer(binary=True)
X = vectorizer.fit_transform(movie_df_new.quote.values)
words = vectorizer.get_feature_names()

In [8]:
# Getting the targer variable
y= movie_df_new['fresh'].values
y = np.where(y == 'fresh', 1, 0)

3. Split your work data and target (i.e. the variable fresh) into training and validation chunks (80/20 or so).

In [9]:
# Splitting the data
X_train,X_valid, y_train, y_valid = train_test_split(X,y, test_size = 0.2, random_state=0)

Good. Now you are ready with the preparatory work and it's time to dive into the real thing. Let'simplement Naive Bayes. Use only training data when doing the fitting below.

4. Compute the unconditional (log) probability that the tomato is fresh/rotten, log Pr(F), and log Pr(R). These probabilities are based on the values of fresh alone, not on the words the quotes contain.

In [10]:
# Calculating the logPr(F)
log_pr_f = np.log(y_train[y_train==1].shape[0]/y_train.shape[0])
log_pr_f

-0.4766154175778479

In [11]:
# Calculating the logPr(R)
log_pr_r = np.log(y_train[y_train==0].shape[0]/y_train.shape[0])
log_pr_r

-0.9699058337934665

5. For each word w, compute log Pr(wjF) and log Pr(wjR), the (log) probability that the word is present in a fresh/rotten review. These probabilities can easily be calculated from counts of how many times these words are present for each class.

In [12]:
fresh = y_train[y_train==1].shape[0]
rotten = y_train[y_train==0].shape[0]

In [13]:
index = np.array(range(len(y_train)))
fresh_index = []
rotten_index = []
for i in index:
    if y_train[i] == 0:
        rotten_index.append(i)
    if y_train[i] == 1:
        fresh_index.append(i)

In [14]:
fresh_words  = X_train[fresh_index]
rotten_words = X_train[rotten_index]

In [15]:
pr_w_f = np.sum(fresh_words, axis =0 )/ fresh 
pr_w_r = np.sum(rotten_words, axis = 0) / rotten 

In [16]:
pr_w_f = pd.Series(np.array(pr_w_f).ravel())
pr_w_r = pd.Series(np.array(pr_w_r).ravel())

Calculating the log probability that the word is present in a fresh/rotten review.

In [17]:
log_pr_w_f = pr_w_f.apply(lambda x: np.log(x) if x!=0 else 0)
log_pr_w_r = pr_w_r.apply(lambda x: np.log(x) if x!=0 else 0)

6. For both destination classes, F and R, compute the log-likelihood that the quote belongs to this class. Computing these likelihoods involves sums of the previously computed probabilities, log Pr(wjF), and BOW elements xij. Check out np.apply_along_axis that can be used to apply a function on matrix columns/rows so you can create a fairly good one-liner to compute log-likelihood.

In [18]:
X_valid_vectorized = pd.DataFrame(X_valid.toarray())

In [19]:
prediction = []

def log_likelihood(ith_review):
    log_fresh_sum = 0
    log_rotten_sum = 0
   
    for j in range(ith_review.shape[0]):
        if (ith_review[j]!=0):

            log_fresh_sum =  log_fresh_sum + (log_pr_w_f[j])
            log_rotten_sum =  log_rotten_sum + ((log_pr_w_r[j]))

    log_likelihood_fresh_ith_review = log_fresh_sum + log_pr_f
    log_likelihood_rotten_ith_review = log_rotten_sum + log_pr_r
    
    if  log_likelihood_fresh_ith_review > log_likelihood_rotten_ith_review:
        prediction.append(1)
    else:
        prediction.append(0)   

In [20]:
np.apply_along_axis(log_likelihood, 1, X_valid_vectorized )

array([None, None, None, ..., None, None, None], dtype=object)

7. Print the resulting confusion matrix and accuracy (feel free to use existing libraries).

In [21]:
pred = np.array(prediction)
print("Confusion matrix:")
print( confusion_matrix(y_valid ,pred))
print("Accuracy of the classifier is" , round((accuracy_score(y_valid, pred)*100),2),"%")

Confusion matrix:
[[624 355]
 [976 610]]
Accuracy of the classifier is 48.11 %


# 3 Interpretation

Now it is time to look at your fitted model a little bit closer. NB model probabilities are rather easy to understand and interpret. The task here is to and the best words to predict a fresh, and a rotten review. And we only want to look at words that are reasonably frequent, say more frequent than 30 times in the data.

1. Extract from your conditional probability vectors log Pr(wjF) and log Pr(wjR) the probabilities that correspond to frequent words only.

In [22]:
sum_of_occurences_of_each_word = pd.DataFrame(X_train.sum(axis=0))
#selecting words with frequency greater than 30
sum_of_occurences_of_each_word=sum_of_occurences_of_each_word.ix[:,sum_of_occurences_of_each_word.gt(31).any()] 
sum_of_occurences_of_each_word=sum_of_occurences_of_each_word.transpose()

In [23]:
words=pd.DataFrame(words)
log_pr_w_f_df=pd.DataFrame(log_pr_w_f)
log_pr_w_r_df=pd.DataFrame(log_pr_w_r)
new_tab=pd.concat([words,sum_of_occurences_of_each_word,log_pr_w_f_df,log_pr_w_r_df], axis=1)
new_tab=new_tab.dropna()
new_tab.columns=["Word","Word Frequency","log_pr_w_f", "log_pr_w_r"]
new_tab["Difference"]=log_pr_w_f_df-log_pr_w_r_df
new_tab

,Word,Word Frequency,log_pr_w_f,log_pr_w_r,Difference
265,about,652.0,-2.805954,-2.678659,-0.127296
266,above,41.0,-5.463361,-5.626850,0.163489
352,achievement,32.0,-5.426993,-6.879613,1.452620
376,across,32.0,-6.194248,-5.321468,-0.872780
377,act,37.0,-5.986609,-5.221385,-0.765224
...,...,...,...,...,...
20767,years,113.0,-4.215903,-5.321468,1.105565
20775,yet,130.0,-4.281861,-4.528238,0.246377
20792,you,677.0,-2.728512,-2.701387,-0.027126
20793,young,86.0,-4.648324,-5.047032,0.398708


2. Find 10 best words to predict F and 10 best words to predict R. Hint: imagine we have a review that contains just a single word. Which word will give the highest weight to the probability the review is fresh? Which one to the likelihood it is rotten?Comment your results.

Answer:

From the below table, we can see that words like "masterpiece", "touching", "extraordinary" etc. are the best words to predict a fresh review as for these words, the probability of them being fresh is large and at the same time, the probability of them being rotten is small (which can be seen as the difference between log Pr(w|F) and log Pr(w|R) is the largest for these words). Also, logically these words have a positive sentiment and would surely be present in a fresh review.

In [24]:
new_tab.sort_values(by=['Difference'], ascending=False).head(10)

,Word,Word Frequency,log_pr_w_f,log_pr_w_r,Difference
11319,masterpiece,54.0,-4.807954,-7.572760,2.764806
18919,touching,37.0,-5.203850,-7.572760,2.368910
6525,extraordinary,36.0,-5.232837,-7.572760,2.339923
14821,recent,43.0,-5.070318,-7.167295,2.096977
3666,complex,39.0,-5.175679,-7.167295,1.991616
20595,witty,49.0,-4.952535,-6.879613,1.927078
17913,succeeds,34.0,-5.325211,-7.167295,1.842084
10307,kubrick,34.0,-5.325211,-7.167295,1.842084
7494,funniest,33.0,-5.358000,-7.167295,1.809295
8062,greatest,42.0,-5.121612,-6.879613,1.758001


Answer:

From the below table, we can see that words like "mess, "worst", "lacks", "dull" etc. are the best words to predict a rotten review as for these words, the probability of them being rotten is large and at the same time, the probability of them being fresh is small (which can be seen as the difference between log Pr(w|F) and log Pr(w|R) is the smallest for these words). Also, logically these words have a negative sentiment and would surely be present in a rotten review.

In [25]:
new_tab.sort_values(by=['Difference'], ascending=False).tail(10)

,Word,Word Frequency,log_pr_w_f,log_pr_w_r,Difference
12285,needs,39.0,-6.456613,-4.898612,-1.558001
11573,mess,40.0,-6.456613,-4.864710,-1.591903
12452,nor,39.0,-6.561973,-4.864710,-1.697263
20676,worst,44.0,-6.456613,-4.739547,-1.717066
10347,lacks,44.0,-6.561973,-4.710559,-1.851414
7032,flat,41.0,-6.679756,-4.769400,-1.910356
14188,problem,39.0,-6.813288,-4.800171,-2.013116
6593,fails,45.0,-6.679756,-4.654989,-2.024767
5600,dull,34.0,-7.372903,-4.864710,-2.508193
19543,unfortunately,36.0,-8.759198,-4.710559,-4.048638


3. Print out a few missclassified quotes. Can you understand why these are misclassified?

Below we can see that a few quotes are misclassified between the validation set and the predicted result

In [26]:
misclass=(y_valid&pred)
print(misclass[1:10])

[0 1 0 1 0 0 0 1 0]


In [27]:
y = movie_df_new['fresh']
X_train,X_valid, y_train, y_valid = train_test_split(X,y, test_size = 0.2, random_state=0)
print(y_valid[5:7])
print(y_valid[8:9])

3616     fresh
2188    rotten
Name: fresh, dtype: object
6489    fresh
Name: fresh, dtype: object


Misclassified quote 1:

The below fresh review was misclassified as rotten probably because it has a strong negative word like "wrong" which has a higher probability of being present in rotten reviews.

In [28]:
movie_df_new[movie_df_new.index==3616]

,critic,fresh,imdb,link,publication,quote,review_date,rtid,title
3616,J. Hoberman,fresh,53125,http://www.villagevoice.com/issues/9942/hoberm...,Village Voice,Hitchcock's ultimate wrong-man comedy.,2000-01-01 00:00:00,18639,North by Northwest


Misclassified quote 2:

The below fresh review was misclassified as rotten probably because it has negative words like "never" and "unpredictable" which have a higher probability of being present in rotten reviews.

In [29]:
movie_df_new[movie_df_new.index==6489]

,critic,fresh,imdb,link,publication,quote,review_date,rtid,title
6489,Roger Ebert,fresh,119738,http://www.rogerebert.com/reviews/my-best-frie...,Chicago Sun-Times,The screenplay has never been on autopilot; it...,2000-01-01 00:00:00,12712,My Best Friend's Wedding


Misclassified quote 3:

The below rotten review was misclassified as fresh probably because it has a positive word like "heaven" which has a higher probability of being present in fresh reviews and also the other words in this review are neutral and do not have a high probability of being strongly fresh or rotten.

In [30]:
movie_df_new[movie_df_new.index==2188]

,critic,fresh,imdb,link,publication,quote,review_date,rtid,title
2188,Desson Thomson,rotten,107096,http://www.washingtonpost.com/wp-srv/style/lon...,Washington Post,"Heaven has so many themes, ranging from Buddhi...",2001-11-16 00:00:00,14946,Heaven & Earth


Thus, the presence of stop words in reviews such as "the", "in", "or", "and" etc. do not specifically lend themselves to fresh or rotten reviews and have an affect on the correct classification of revies. Also, words which are present with a low frequency in the reviews cannot be highly weighted as fresh or rotten and can lead to the review being misclassified. Lastly, if a review contains many unseen words on which the model has not been trained on then it can also lead to the review being misclassified.

## 4 NB with smoothing

1. Create two functions: one for fitting NB model, and another to predict outcome based on the fitted model. As mentioned above, the model is fully described with 4 probabilities, so your fitting function may return such a list as the model; and the prediction function may take it as an input.

2. Add smoothing to the model.

In [31]:
y=movie_df_new['fresh'].values
y=np.where(y=="fresh",1,0)
#X_train,X_valid, y_train, y_valid = train_test_split(X,y, test_size = 0.2, random_state=0)

In [32]:
#NB fitting model with smoothening

def fit_model(X_train, y_train, alpha):
    log_pr_f = np.log(len(y_train[y_train==1])/len(y_train))
    log_pr_r = np.log(len(y_train[y_train==0])/len(y_train))
    
    count_fresh = len(y_train[y_train==1])
    count_rotten = len(y_train[y_train==0])
    
    index = np.array(range(len(y_train)))
    fresh_index = []
    rotten_index = []
    for i in index:
        if y_train[i] == 0:
            rotten_index.append(i)
        if y_train[i] == 1:
            fresh_index.append(i)
            
    fresh_words  = X_train[fresh_index]
    rotten_words = X_train[rotten_index]
    
    fresh_sum  = np.sum(fresh_words,  axis = 0 )
    rotten_sum = np.sum(rotten_words, axis = 0)
    
    pr_w_f = (fresh_sum + alpha)/ (count_fresh + (alpha)) #smoothening the model
    pr_w_r = (rotten_sum + alpha)/ (count_rotten + (alpha)) #smoothening the model
    
    pr_w_f = pd.Series(np.array(pr_w_f).ravel())
    pr_w_r = pd.Series(np.array(pr_w_r).ravel())
    
    log_pr_w_f = pr_w_f.apply(lambda x: np.log(x) if x!=0 else 0)
    log_pr_w_r = pr_w_r.apply(lambda x: np.log(x) if x!=0 else 0)
    
    return log_pr_f, log_pr_r, log_pr_w_f, log_pr_w_r

In [33]:
#Function to predict the outcome

def predict_outcome(log_pr_f, log_pr_r, log_pr_w_f, log_pr_w_r, X_valid):
    
    X_valid_vectorized = pd.DataFrame(X_valid.toarray())
    np.apply_along_axis(log_likelihood, 1, X_valid_vectorized)   


3. Cross-validate the accuracy (on the validation data) on a number of alpha values and and the alpha that gives you the best result.

In [34]:
scores = []

cv = KFold(n_splits=10, random_state=42, shuffle=False)

for alpha in [0.05,0.2,0.3,0.5,0.7,0.9]:
    
    scores_cv = []
    
    for train_index, test_index in cv.split(X):

        X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]
        
        global log_pr_f
        global log_pr_r
        global log_pr_w_f
        global log_pr_w_r 
        global prediction
        prediction = []
        log_pr_f, log_pr_r, log_pr_w_f, log_pr_w_r = fit_model(X_train, y_train, alpha = alpha)
        predict_outcome(log_pr_f, log_pr_r, log_pr_w_f, log_pr_w_r, X_test)
        pred = np.array(prediction)
        scores_cv.append(accuracy_score(y_test, pred))
        
    scores.append(np.mean(np.array(scores_cv)))

In [35]:
alpha = [0.05,0.2,0.3,0.5,0.7,0.9]
for i in range(len(scores)):
    print("The accuracy of the model is" , round((scores[i]*100),2),"%", "when alpha is",alpha[i])

The accuracy of the model is 73.15 % when alpha is 0.05
The accuracy of the model is 74.12 % when alpha is 0.2
The accuracy of the model is 74.26 % when alpha is 0.3
The accuracy of the model is 74.2 % when alpha is 0.5
The accuracy of the model is 73.86 % when alpha is 0.7
The accuracy of the model is 73.39 % when alpha is 0.9


Thus, we can see that when alpha is 0.3 our model has the best accuracy of 74.26%.